In [2]:
import csv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2022-12-04 15:00:46.537790: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-04 15:00:46.751022: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-04 15:00:47.487161: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-04 15:00:47.487210: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

Num GPUs Available:  0


2022-12-04 15:00:49.316475: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2022-12-04 15:00:49.316515: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: danilo-Nitro-AN515-58
2022-12-04 15:00:49.316520: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: danilo-Nitro-AN515-58
2022-12-04 15:00:49.316636: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.108.3
2022-12-04 15:00:49.316656: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.60.11
2022-12-04 15:00:49.316659: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 525.60.11 does not match DSO version 510.108.3 -- cannot find working devices in this co

In [4]:
train_data = pd.read_csv("data/train.csv")

In [5]:
# new features
train_data["url_count"] = train_data["urls"].apply(lambda s: s[1:-1].count("\'")/2)
train_data["text_len"] = train_data["text"].apply(lambda s: len(s))
train_data["hashtags_count"] = train_data["hashtags"].apply(lambda s: s[1:-1].count("\'")/2)
train_data["day"] = train_data["timestamp"].apply(lambda t: datetime.utcfromtimestamp(t/1000).day)
train_data["hour"] = train_data["timestamp"].apply(lambda t: datetime.utcfromtimestamp(t/1000).hour)

# indicators of keywords
train_data["Macron"] =  train_data["text"].apply(lambda s: ("macron" in s.lower().split()))
train_data["Zemmour"] =  train_data["text"].apply(lambda s: ("zemmour" in s.lower().split()))
train_data["Melenchon"] =  train_data["text"].apply(lambda s: ("melenchon" in s.replace("é","e").lower().split()))
train_data["rt"] =  train_data["text"].apply(lambda s: ("rt" in s.lower().split()))

In [6]:
# select useful columns
train_data_filtered = train_data.drop(["text", "urls", "mentions", "hashtags", "timestamp", "TweetID"], axis=1)

In [7]:
display(train_data_filtered)

,retweets_count,favorites_count,followers_count,statuses_count,friends_count,verified,url_count,text_len,hashtags_count,day,hour,Macron,Zemmour,Melenchon,rt
0,3,0,3682,453535,3628,0,0.0,34,0.0,11,5,True,False,False,True
1,0,0,86,1016,284,0,0.0,9,0.0,19,12,False,False,False,False
2,3,1,1944,28234,1995,0,0.0,19,0.0,15,18,False,False,False,False
3,0,0,1,1072,0,0,1.0,123,0.0,14,11,False,False,False,False
4,0,0,13957,25311,10841,0,0.0,26,0.0,14,11,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353964,0,0,34,1509,55,0,1.0,30,0.0,16,13,False,False,False,False
353965,0,0,89,11166,127,0,0.0,69,0.0,12,8,False,False,False,False
353966,3,0,1888,712,3086,0,0.0,29,0.0,18,12,False,True,False,False
353967,0,0,139,486,320,0,0.0,24,0.0,11,8,False,False,False,False


In [8]:
emb_matrix = np.load("data/train_emb_matrix.npy")
print(train_data_filtered.shape, emb_matrix.shape)

(353969, 15) (353969, 25, 16)


In [9]:
# TODO: normalizar colunas embeddings

In [10]:
data_newdim = np.expand_dims(np.array(train_data_filtered), axis=1)
data_reshaped = np.broadcast_to(data_newdim, (emb_matrix.shape[0], emb_matrix.shape[1], train_data_filtered.shape[1]))
data_reshaped[0][:2]

array([[3, 0, 3682, 453535, 3628, 0, 0.0, 34, 0.0, 11, 5, True, False,
        False, True],
       [3, 0, 3682, 453535, 3628, 0, 0.0, 34, 0.0, 11, 5, True, False,
        False, True],
       [3, 0, 3682, 453535, 3628, 0, 0.0, 34, 0.0, 11, 5, True, False,
        False, True],
       [3, 0, 3682, 453535, 3628, 0, 0.0, 34, 0.0, 11, 5, True, False,
        False, True],
       [3, 0, 3682, 453535, 3628, 0, 0.0, 34, 0.0, 11, 5, True, False,
        False, True]], dtype=object)

In [11]:
full_matrix = np.concatenate((data_reshaped, emb_matrix), axis=2)

In [13]:
print(full_matrix[0][:2])

[[3 0 3682 453535 3628 0 0.0 34 0.0 11 5 True False False True
  0.18788284063339233 -0.5350038409233093 -1.7558887004852295
  0.6010056138038635 -0.38633257150650024 0.23139137029647827
  -0.8143761157989502 0.6429756879806519 0.07032456994056702
  0.047739773988723755 -1.0532739162445068 -0.04242870211601257
  0.4343041181564331 -0.9819753766059875 -0.49964916706085205
  1.6986557245254517]
 [3 0 3682 453535 3628 0 0.0 34 0.0 11 5 True False False True
  0.37392178177833557 1.6400281190872192 -0.06540881097316742
  -0.5598422288894653 -0.7594510316848755 -0.1807458996772766
  -0.2969696819782257 0.3544090688228607 -0.4949268400669098
  0.5475368499755859 0.7976458668708801 -0.10759757459163666
  0.3659772574901581 -0.044347986578941345 -0.588219165802002
  -0.4794001579284668]]


In [ ]:
# select useful columns
train_data_filtered = train_data.drop(["text", "urls", "mentions", "hashtags", "timestamp", "TweetID"], axis=1)
# train_data_filtered = train_data.loc[:, ["retweets_count","favorites_count","followers_count","statuses_count","friends_count",
#                                  "hashtags_count","hour","verified","url_count","text_len","rt","Macron","Zemmour","Melenchon"]]


X_train, X_eval, y_train, y_eval = train_test_split(train_data_filtered.drop("retweets_count", axis=1),
                                                    train_data_filtered["retweets_count"],
                                                    random_state=42, test_size=0.1)

# Standardize the data
normal_columns = train_data_filtered.drop(["hour", "verified", "Macron", "Zemmour", "Melenchon", "url_count", "rt", "retweets_count"], axis=1).columns
mu, sigma = X_train[normal_columns].mean(axis=0), X_train[normal_columns].std(axis=0)
X_train.loc[:, normal_columns] = (X_train[normal_columns] - mu) / sigma
X_eval.loc[:, normal_columns] = (X_eval[normal_columns] - mu) / sigma

In [ ]:
import tensorflow as tf

In [ ]:
# model = tf.keras.models.Sequential([
#   tf.keras.layers.Dense(64, activation='relu'),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(1),
# ])

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional

n_timesteps = 40
input_features = X_train.shape[1]

model = Sequential()
model.add(Bidirectional(LSTM(30, return_sequences=True), input_shape=(n_timesteps, input_features)))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))

In [ ]:
# import tensorflow_addons as tfa
# optimizer = tfa.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-4)
# model.compile(optimizer=optimizer, loss='mae')

In [ ]:
model.compile(optimizer="adam", loss='mae')

In [ ]:
history = model.fit(X_train.values.astype(np.float32), y_train.values.astype(np.float32), epochs=150, batch_size=256,
         validation_data=(X_eval.values.astype(np.float32), y_eval.values.astype(np.float32)), shuffle=True)

In [ ]:
plt.plot(history.epoch, history.history["loss"], 'b', label='Training loss')
plt.plot(history.epoch, history.history["val_loss"], 'g', label='Validation loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model.evaluate(X_eval.values.astype(np.float32),  y_eval.values.astype(np.float32), verbose=2)

In [ ]:
pred = model.predict(X_train.values.astype(np.float32))
print(pred)

In [ ]:
# plt.scatter(X_train["favorites_count"], model.predict(X_train))
# plt.scatter(X_train["favorites_count"], y_train.values.astype(np.float32))

In [ ]:
# evaluation

eval_data = pd.read_csv("data/evaluation.csv")
tweets = eval_data["TweetID"]

In [ ]:
eval_data["url_count"] = eval_data["urls"].apply(lambda s: s[1:-1].count("\'")/2)
eval_data["text_len"] = eval_data["text"].apply(lambda s: len(s))
eval_data["hashtags_count"] = eval_data["hashtags"].apply(lambda s: s[1:-1].count("\'")/2)
eval_data["day"] = eval_data["timestamp"].apply(lambda t: datetime.utcfromtimestamp(t/1000).day)
eval_data["hour"] = eval_data["timestamp"].apply(lambda t: datetime.utcfromtimestamp(t/1000).hour)
eval_data["Macron"] =  eval_data["text"].apply(lambda s: ("macron" in s.lower().split()))
eval_data["Zemmour"] =  eval_data["text"].apply(lambda s: ("zemmour" in s.lower().split()))
eval_data["Melenchon"] =  eval_data["text"].apply(lambda s: ("melenchon" in s.lower().split()))
eval_data["rt"] =  eval_data["text"].apply(lambda s: ("rt" in s.lower().split()))

# print("sentiment analysis...")
# eval_data["compound"] =  eval_data["text"].apply(lambda s: sia.polarity_scores(s)['compound'])

eval_data = eval_data.drop(["text", "urls", "mentions", "hashtags", "timestamp", "TweetID"], axis=1)

# normalize
eval_data.loc[:, normal_columns] = (eval_data.loc[:, normal_columns] - mu) / sigma

print(eval_data)

pred = model.predict(eval_data.values.astype(np.float32))

print(pred)

# output normalization
for i,p in enumerate(pred):
    if p<0: pred[i] = 0

In [ ]:
with open("data/predictions.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["TweetID", "retweets_count"])
    for index, prediction in enumerate(pred):
        writer.writerow([str(tweets[index]) , str(int(prediction))])